In [92]:
# import libraries
# !pip install kaggle
# !pip install pandas

import kaggle
import zipfile
import pandas as pd
import sqlalchemy as sal

In [15]:
# download dataset using kaggle API
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [82]:
# extract zip file

zipfile_name = 'orders.csv.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file: # loading the file
    file.extractall() # extract the file

In [83]:
# read data from the file and handle null values

df = pd.read_csv('orders.csv', na_values=['Not Available', 'unknown'])

In [84]:
# rename columns name

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

In [85]:
# derive new columns discount, sales price and profit

df['discount'] = df['list_price']*df['discount_percent']/100
df['sales_price'] = df['list_price'] - df['discount']
df['profit'] = df['sales_price'] - df['cost_price']

In [86]:
#  comvert order date from object data type to datetime
pd.to_datetime(df['order_date'], format="%Y-%m-%d")

0      2023-03-01
1      2023-08-15
2      2023-01-10
3      2022-06-18
4      2022-07-13
          ...    
9989   2023-02-18
9990   2023-03-17
9991   2022-08-07
9992   2022-11-19
9993   2022-07-17
Name: order_date, Length: 9994, dtype: datetime64[ns]

In [87]:
# drop cost price, list price and discount percent columns
df.drop(columns=['list_price', 'cost_price', 'discount_percent'],inplace=True)

In [95]:
# load the data into sql server
engine = sal.create_engine('mssql://Pat/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
connection=engine.connect()
df.to_sql('df_orders', con=connection, index=False, if_exists = 'replace')

38